In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, regexp_replace, trim, length, when, expr, monotonically_increasing_id

# Создание Spark сессии с указанием зависимости и параметрами памяти
spark = SparkSession.builder \
    .appName("Excel to PySpark") \
    .config("spark.jars.packages", "com.crealytics:spark-excel_2.12:0.14.0") \
    .config("spark.master", "local") \
    .config('spark.driver.memory', '2g') \
    .config('spark.executor.cores', '4') \
    .config('spark.executor.memory', '4g') \
    .getOrCreate()

# Загрузка первого листа из Excel
df = spark.read.format("com.crealytics.spark.excel") \
    .option("header", "true") \
    .option("dataAddress", "Данные!A3") \
    .load("/home/jovyan/work/PySpark_test/config/Численность выбывших.xls")

# Кешируем df после загрузки
df = df.cache()

df = df.withColumnRenamed("Классификатор объектов административно-территориального деления (ОКАТО)", "region") \
    .withColumnRenamed("Классификатор стран мира ", "world") \
    .withColumnRenamed("Единица измерения", "unit_name") \
    .withColumnRenamed("Период", "period_name") \
    .withColumnRenamed("Потоки миграции", "migration")

# Обработка region
df = df.withColumn("region", trim(col("region"))) \
    .withColumn("region_okato", split(col("region"), " ").getItem(0)) \
    .withColumn("region", expr("substring(region, instr(region, ' ') + 1)")) \
    .filter(length(col("region_okato")) == 11)

# Обработка unit_name, period_name, migration, world
df = df.withColumn("unit_name", trim(regexp_replace(col("unit_name"), "\\d+", ""))) \
    .withColumn("period_name", trim(regexp_replace(col("period_name"), "\\d+", ""))) \
    .withColumn("migration", trim(regexp_replace(col("migration"), "\\d+", ""))) \
    .withColumn("world", trim(regexp_replace(col("world"), "\\d+", "")))

months = ['январь', 'февраль', 'март', 'апрель', 'май', 'июнь', 'июль', 'август', 'сентябрь', 'октябрь', 'ноябрь', 'декабрь']
df = df.filter(col("period_name").isin(months))

years = [str(year) for year in range(1993, 2024)]

stack_expr = "stack({}, {}) as (years, leaving)".format(len(years), ', '.join([f"'{year}', `{year}`" for year in years]))

df_melted = df.selectExpr('*', stack_expr).drop(*years)

# Кешируем df_melted для последующих операций
df_melted = df_melted.cache()

# Подготовка итогового DataFrame
df_melted = df_melted.withColumn("leaving", when(col("leaving") == "", 0).otherwise(col("leaving"))) \
    .withColumn("years", col("years").cast("int")) \
    .withColumn("leaving", col("leaving").cast("int"))

df_melted = df_melted.withColumn(
    "leaving", when((col("leaving") == "") | (col("leaving").isNull()), 0).otherwise(col("leaving"))) \
    .withColumn("years", col("years").cast("int")) \
    .withColumn("leaving", col("leaving").cast("int"))

df_melted = df_melted.filter(df_melted.leaving > 0)

df_melted = df_melted.select("region", "region_okato", "world", "unit_name", "period_name", "migration", "years", "leaving")

# Сохранение итогового DataFrame в CSV
output_path = "/home/jovyan/work/PySpark_test/config/result.csv"
df_melted.coalesce(1).write.csv(output_path, header=True, mode="overwrite")

# Закрытие Spark сессии
spark.stop()

Активные Spark сессии: http://b35f097c8a09:4040


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, regexp_replace, trim, length, when, expr, monotonically_increasing_id

# Создание Spark сессии с указанием зависимости и параметрами памяти
spark = SparkSession.builder \
    .appName("Excel to PySpark") \
    .config("spark.jars.packages", "com.crealytics:spark-excel_2.12:0.14.0") \
    .config("spark.master", "local") \
    .config('spark.driver.memory', '4g') \
    .config('spark.executor.cores', '4') \
    .config('spark.executor.memory', '8g') \
    .getOrCreate()

sheet_names = ["data1", "data2", "data3", "data4"]
df = None

for sheet in sheet_names:
    # Загрузка текущего листа
    combined_df = spark.read.format("com.crealytics.spark.excel") \
        .option("header", "true") \
        .option("dataAddress", f"{sheet}!A3") \
        .load("/home/jovyan/work/PySpark_test/config/Численность выбывших.xls")

    if df is None:
        df = combined_df
    else:
        df = df.union(combined_df)

In [2]:
# Кешируем df после загрузки
df = df.cache()

df = df.withColumnRenamed("Классификатор объектов административно-территориального деления (ОКАТО)", "region") \
    .withColumnRenamed("Классификатор стран мира ", "world") \
    .withColumnRenamed("Единица измерения", "unit_name") \
    .withColumnRenamed("Период", "period_name") \
    .withColumnRenamed("Потоки миграции", "migration")

# Обработка region
df = df.withColumn("region", trim(col("region"))) \
    .withColumn("region_okato", split(col("region"), " ").getItem(0)) \
    .withColumn("region", expr("substring(region, instr(region, ' ') + 1)")) \
    .filter(length(col("region_okato")) == 11)

# Обработка unit_name, period_name, migration, world
df = df.withColumn("unit_name", trim(regexp_replace(col("unit_name"), "\\d+", ""))) \
    .withColumn("period_name", trim(regexp_replace(col("period_name"), "\\d+", ""))) \
    .withColumn("migration", trim(regexp_replace(col("migration"), "\\d+", ""))) \
    .withColumn("world", trim(regexp_replace(col("world"), "\\d+", "")))

months = ['январь', 'февраль', 'март', 'апрель', 'май', 'июнь', 'июль', 'август', 'сентябрь', 'октябрь', 'ноябрь', 'декабрь']
df = df.filter(col("period_name").isin(months))

years = [str(year) for year in range(1993, 2024)]

stack_expr = "stack({}, {}) as (years, leaving)".format(len(years), ', '.join([f"'{year}', `{year}`" for year in years]))

df_melted = df.selectExpr('*', stack_expr).drop(*years)

# Кешируем df_melted для последующих операций
df_melted = df_melted.cache()

# Подготовка итогового DataFrame
df_melted = df_melted.withColumn("leaving", when(col("leaving") == "", 0).otherwise(col("leaving"))) \
    .withColumn("years", col("years").cast("int")) \
    .withColumn("leaving", col("leaving").cast("int"))

df_melted = df_melted.withColumn(
    "leaving", when((col("leaving") == "") | (col("leaving").isNull()), 0).otherwise(col("leaving"))) \
    .withColumn("years", col("years").cast("int")) \
    .withColumn("leaving", col("leaving").cast("int"))

df_melted = df_melted.filter(df_melted.leaving > 0)

df_melted = df_melted.select("region", "region_okato", "world", "unit_name", "period_name", "migration", "years", "leaving")

# Сохранение итогового DataFrame в CSV
output_path = "/home/jovyan/work/PySpark_test/config/result.csv"
df_melted.coalesce(1).write.csv(output_path, header=True, mode="overwrite")

# Закрытие Spark сессии
spark.stop()